# Create CORD19 MongoDB Database Entries

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/covid-paper-browser/blob/master/notebooks/create_db_entries_pickles.ipynb)

This notebook is intended to be run on Google Colab or other GPU providers platforms as a step-by-step replacement for the `create_db` script that can be run locally and don't require excessive amounts of time.

**Disclaimer**: The procedure requires having access to a Google Drive account with at least 13 GB of storage available. The total runtime in Colab is approximately 2 h 45 minutes.

## Steps

1. Open this notebook in Colab by clicking the button above.

2. Connect to a GPU runtime and mount your Drive (on the left, Files -> Mount Drive). You should see the path `drive/My Drive` containing your files on the left section.

3. Run the cells below. Those will download data, install Python libraries, define methods and ultimately run the `create_database` method. Instead of actually creating the database, this saves the database entries in `X` files named `db_entriesX.pkl` on your Google Drive. The total size is approx 12.45 GB for the full entries (including both `title_abstract_embeddings` and `paragraphs_embeddings`), or 2.3 GB if you decide to compute only `title_abstract_embeddings`. The latter are also way faster (11 min 40 sec for me).

4. Download the pickled files on your PC.

5. Run a MongoDB session in the background, open a Python session in the same folder of the downloaded files and run:

```python
import os
from pymongo import MongoClient

YOUR_DB_NAME = 'coviddb'
YOUR_COLLECTION_NAME = 'cord19scibert'

files = [f for f in os.listdir() if f.startswith('db_entries')]
client = MongoClient()
db = client[YOUR_DB_NAME]
col = db[YOUR_COLLECTION_NAME]
for fname in tqdm(files):
    with open(fname, 'rb') as f:
        entries = pickle.load(f)
    col.insert_many(entries)
```

Notice that doing this may cannibalize your resources (it did with me on a 16GB RAM machine), so consider doing this by loading and inserting one pickled file at a time.


## Download Data from AI2 Servers

In [0]:
%%bash capture
mkdir data

DATE=2020-03-27
DATA_DIR=data

wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/"${DATE}"/comm_use_subset.tar.gz -P "${DATA_DIR}"
wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/"${DATE}"/noncomm_use_subset.tar.gz -P "${DATA_DIR}"
wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/"${DATE}"/custom_license.tar.gz -P "${DATA_DIR}"
wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/"${DATE}"/biorxiv_medrxiv.tar.gz -P "${DATA_DIR}"
wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/"${DATE}"/metadata.csv -P "${DATA_DIR}"

tar -zxvf "${DATA_DIR}"/comm_use_subset.tar.gz -C "${DATA_DIR}"
tar -zxvf "${DATA_DIR}"/noncomm_use_subset.tar.gz -C "${DATA_DIR}"
tar -zxvf "${DATA_DIR}"/custom_license.tar.gz -C "${DATA_DIR}"
tar -zxvf "${DATA_DIR}"/biorxiv_medrxiv.tar.gz -C "${DATA_DIR}"

## Install Python Libraries

In [0]:
%%capture
!pip install -U transformers pandas sentence_transformers tqdm compress_pickle

## Define the PaperDatabaseEntry Class

In [0]:
class PaperDatabaseEntry:
    """ Defines the Paper object stored in the database. """
    def __init__(self, x):
        self.cord_id = x['cord_uid']
        self.url = x['url']
        self.sha = x['sha'].split(';')[0]
        self.title = x['title'] if x['title'] not in FILTER_TITLES else ''
        self.source = x['source_x']
        self.doi = x['doi']
        self.pmc_id = x['pmcid']
        self.pubmed_id = x['pubmed_id']
        self.license = x['license']
        self.abstract = x['abstract'] if x['abstract'] not in FILTER_ABSTRACTS else ''
        self.publish_time = x['publish_time']
        self.authors = x['authors'].split('; ')
        self.journal = x['journal']
        self.microsoft_id = x['Microsoft Academic Paper ID']
        self.who_id = x['WHO #Covidence']
        self.paragraphs = [] # List of tuples (section_name, text)
        self.bibliography = [] # List of dictionaries
        self.title_abstract_embeddings = []
        self.paragraphs_embeddings = []

    def as_dict(self):
        return {
            'cord_id': self.cord_id,
            'url': self.url,
            'sha': self.sha,
            'title': self.title,
            'source': self.source,
            'doi': self.doi,
            'pmc_id': self.pmc_id,
            'pubmed_id': self.pubmed_id,
            'license': self.license,
            'abstract': self.abstract,
            'publish_time': self.publish_time,
            'authors': self.authors,
            'journal': self.journal,
            'microsoft_id': self.microsoft_id,
            'who_id': self.who_id,
            'paragraphs': self.paragraphs,
            'bibliography': self.bibliography,
            'title_abstract_embeddings': self.title_abstract_embeddings,
            'paragraphs_embeddings': self.paragraphs_embeddings,
        }

    def compute_title_abstract_embeddings(self, model):
        if self.title != '' or self.abstract != '':
            title_abstract = self.title + ' ' + self.abstract
            embedding = model.encode([title_abstract], show_progress_bar=False)
            self.title_abstract_embeddings = embedding[0].tolist()
    
    def compute_paragraphs_embeddings(self, model):
        if len(self.paragraphs) > 0:
            paragraphs_text = [tup[1] for tup in self.paragraphs]
            paragraph_embeddings = model.encode(paragraphs_text, show_progress_bar=False)
            self.paragraphs_embeddings = [e.tolist() for e in paragraph_embeddings]

FILTER_TITLES = ['Index', 'Subject Index', 'Subject index', 'Author index', 'Contents', 
        'Articles of Significant Interest Selected from This Issue by the Editors',
        'Information for Authors', 'Graphical contents list', 'Table of Contents',
        'In brief', 'Preface', 'Editorial Board', 'Author Index',
        'Volume Contents', 'Research brief', 'Abstracts', 'Keyword index',
        'In This Issue', 'Department of Error', 'Contents list', 'Highlights of this issue',
        'Abbreviations', 'Introduction', 'Cumulative Index', 'Positions available',
        'Index of Authors', 'Editorial', 'Journal Watch', 'QUIZ CORNER', 'Foreword', 'Table of contents',
        'Quiz Corner', 'INDEX', 'Bibliography of the current world literature',
        'Index of Subjects', '60 Seconds', 'Contributors',
        'Public Health Watch', 'Commentary', 'Chapter 1 Introduction',
        'Facts and ideas from anywhere', 'Erratum', 'Contents of Volume', 'Patent reports',
        'Oral presentations', 'Abkürzungen', 'Abstracts cont.', 'Related elsevier virology titles contents alert',
        'Keyword Index', 'Volume contents', 'Articles of Significant Interest in This Issue',
        'Appendix', 'Abkürzungsverzeichnis', 'List of Abbreviations', 'Editorial Board and Contents',
        'Instructions for Authors', 'Corrections', 'II. Sachverzeichnis', '1 Introduction',  'List of abbreviations',
        'Response', 'Feedback', 'Poster Sessions', 'News Briefs', 'Commentary on the Feature Article',
        'Papers to Appear in Forthcoming Issues', 'TOC', 'Glossary', 'Letter from the editor', 'Croup',
        'Acronyms and Abbreviations', 'Highlights', 'Forthcoming papers', 'Poster presentations', 'Authors',
        'Journal Roundup', 'Index of authors', 'Table des mots-clés', 'Posters', 'Cumulative Index 2004', 
        'A Message from the Editor', 'Contents and Editorial Board', 'SUBJECT INDEX', 'Contents page 1',
]

FILTER_ABSTRACTS = ['Unknown', '[Image: see text]']

## Methods to Load Model and Rank Abstracts/Paragraphs

In [0]:
import numpy as np
from scipy.spatial.distance import cdist
from sentence_transformers import models, SentenceTransformer


def load_sentence_transformer(
    name: str = 'gsarti/scibert-nli', 
    max_seq_length: int  = 128, 
    do_lower_case: bool  = True) -> SentenceTransformer:
    """ Loads a SentenceTransformer from HuggingFace AutoModel bestiary """
    word_embedding_model = models.BERT(
            'gsarti/scibert-nli',
            max_seq_length=128,
            do_lower_case=True
        )
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
            pooling_mode_mean_tokens=True,
            pooling_mode_cls_token=False,
            pooling_mode_max_tokens=False
        )
    return SentenceTransformer(modules=[word_embedding_model, pooling_model])


def match_query(
    query: str,
    model: SentenceTransformer,
    corpus: list,
    corpus_embed: list,
    top_k: int = 5) -> list:
    """ Matches query and paragraph embeddings, returning top scoring paragraphs ids and scores """
    query_embed = model.encode([query], show_progress_bar=False)[0].reshape(1,-1)
    distances = 1 - cdist(query_embed, corpus_embed, "cosine")
    results = zip(corpus, distances.reshape(-1,1))
    results = sorted(results, key=lambda x: x[1], reverse=True)
    return results[:top_k]

## Methods to Create Database Entries and Pickle Files

In [0]:
import os
import sys
import json
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

def create_db_entry(
    csv_entry: dict, 
    model: SentenceTransformer,
    do_paragraph_embeddings: bool) -> PaperDatabaseEntry:
    """ Creates a single DB entry from a csv entry using the model for creating embeddings """
    db_entry = PaperDatabaseEntry(csv_entry)
    db_entry.compute_title_abstract_embeddings(model)
    if csv_entry['has_full_text'] == True:
        foldername = csv_entry['full_text_file']
        # Format is e.g. 'data/biorxiv_medrxiv/file.json'
        path = os.path.join('data', foldername, f'{db_entry.sha}.json')
        file = json.load(open(path, 'r'))
        paragraphs = []
        # Order is: abstracts, body, back_matter, ref_entries
        parts = [file['abstract'], file['body_text'], file['back_matter']]
        for part in parts:
            for paragraph in part:
                paragraphs.append((paragraph['section'], paragraph['text']))
        for key, paragraph in file['ref_entries'].items():
            paragraphs.append((paragraph['type'].title(), paragraph['text']))
        db_entry.paragraphs = paragraphs
        if do_paragraph_embeddings:
            db_entry.compute_paragraphs_embeddings(model)
        db_entry.bibliography = [file['bib_entries'][entry] for entry in file['bib_entries']]
    return db_entry

def create_pickles(
    model_name: str = 'gsarti/scibert-nli',
    data_path: str = 'data/metadata.csv',
    n_batches: int = 1,
    do_paragraph_embeddings: bool = True) -> None:
    """ Creates a new Mongo database with entries from data_path, using model model_name """
    model = load_sentence_transformer(model_name)
    df = pd.read_csv(data_path)
    df = df.fillna('')
    df_batches = np.array_split(df, n_batches)
    inserted = 0
    for i, batch in enumerate(df_batches):
        if i == 0:
          continue
        print(f'Batch {i}')
        db_entries = []
        for _, row in tqdm(batch.iterrows()):
            db_entry = create_db_entry(row, model, do_paragraph_embeddings)
            # Only add entries with at least one between title and abstract to enable search
            if len(db_entry.title_abstract_embeddings) > 0:
                db_entries.append(db_entry.as_dict())
        print('Saving entries to', f'drive/My Drive/db_entries{i}.pkl')
        with open(f'drive/My Drive/db_entries{i}.pkl', 'wb') as f:
            pickle.dump(db_entries, f)
        inserted += len(db_entries)
        print(f'Inserted {len(db_entries)} new entries.')
    print(f'Done. {len(df)} processed, {inserted} inserted.')

## Run the Creation of Pickle Files

This is the part that will take approximately 3 hours to run.

In [0]:
create_pickles(model=model, data_path='data/metadata.csv', n_batches=5, do_paragraph_embeddings=True)

0it [00:00, ?it/s]

Batch 1


9155it [33:11,  4.60it/s]


Saving entries to drive/My Drive/db_entries1.pkl
Inserted 9147 new entries.
Batch 2


9155it [12:18, 12.40it/s]


Saving entries to drive/My Drive/db_entries2.pkl
Inserted 9125 new entries.
Batch 3


9155it [50:16,  3.03it/s]


Saving entries to drive/My Drive/db_entries3.pkl
Inserted 8918 new entries.
Batch 4


9154it [35:26,  4.31it/s]


Saving entries to drive/My Drive/db_entries4.pkl
Inserted 9100 new entries.
Done. Inserted 45774 total new entries.


## Check that Files were Generated Correctly

You should see roughly 10 GB here

In [3]:
!du -h "drive/My Drive/db_entries"*

1.2G	drive/My Drive/db_entries0.pkl
2.4G	drive/My Drive/db_entries1.pkl
817M	drive/My Drive/db_entries2.pkl
3.7G	drive/My Drive/db_entries3.pkl
2.5G	drive/My Drive/db_entries4.pkl
